# Look for companies with congresspeople names

Sometimes the trade company name is the owner's name, specially when it has just a single parter. Searching for these companies (possibly filtering by legal_entity equals to 213-5 - EMPRESARIO (INDIVIDUAL)) may bring good results.

In [1]:
from serenata_toolbox.datasets import fetch

fetch('2016-12-06-reimbursements.xz', '../data')
fetch('2016-09-03-companies.xz', '../data')
fetch('2016-12-21-deputies.xz', '../data')

In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv('../data/2016-12-06-reimbursements.xz',
                   dtype={'cnpj_cpf': np.str},
                   low_memory=False)

In [3]:
data['supplier'].unique()

array(['MELHOR POSTO DE COMBUSTÍVEIS LTDA', 'MELONGENA PARTICIPAÇÕES LTDA',
       'ARY ANTONIO MOSCHETTA', ..., 'hotel restaurante lancheria do NENE',
       'EDGAR LISBOA - ME', 'ZERO QUINZE'], dtype=object)

In [4]:
data['congressperson_name'].unique()

array(['DILCEU SPERAFICO', 'DOMINGOS DUTRA', 'EDINHO BEZ', ...,
       'MOISÉS DINIZ', 'RICARDO BENTINHO', 'TAMPINHA'], dtype=object)

In [5]:
deputies = pd.read_csv('../data/2016-12-21-deputies.xz',
                       usecols=['congressperson_id', 'civil_name'])
deputies.head()

,congressperson_id,civil_name
0,81366,LUANA MARIA DA SILVA COSTA
1,178980,ANTONIO GOULART DOS REIS
2,141335,JOSÉ ROBERTO OLIVEIRA FARO
3,141531,RODRIGO BATISTA DE CASTRO
4,141478,LAERTE RODRIGUES DE BESSA


Looking for exact matches of names (in supplier names).

In [6]:
import unicodedata

def normalize_string(string):
    if isinstance(string, str):
        nfkd_form = unicodedata.normalize('NFKD', string.lower())
        return nfkd_form.encode('ASCII', 'ignore').decode('utf-8')

In [7]:
data.congressperson_name = data.congressperson_name.apply(normalize_string)
data.supplier = data.supplier.apply(normalize_string)
deputies.civil_name = deputies.civil_name.apply(normalize_string)

In [8]:
supplier_has_congressperson_name = \
    data['supplier'].isin(deputies['civil_name'])
data.loc[supplier_has_congressperson_name,
                   ['issue_date', 'congressperson_name', 'supplier', 'subquota_description', 'cnpj_cpf', 'document_id', 'total_net_value']]

,issue_date,congressperson_name,supplier,subquota_description,cnpj_cpf,document_id,total_net_value
200406,2010-12-28T00:00:00,marcelo castro,marcelo costa e castro,Flight tickets,04114985000137,1985323,12000.00
204127,2010-08-27T00:00:00,paulo magalhaes,paulo sergio paranhos de magalhaes,"Consultancy, research and technical work",06253998000112,1919603,8900.00
233647,2010-07-06T00:00:00,carlos manato,carlos humberto mannato,Telecommunication,04164616000159,1893988,361.92
233652,2010-07-19T00:00:00,carlos manato,carlos humberto mannato,Automotive vehicle renting or watercraft charter,57480788720,1894078,4500.00
295219,2010-01-25T00:00:00,laerte bessa,laerte rodrigues de bessa,Flight tickets,07575651000159,1763526,667.52
295220,2010-01-29T00:00:00,laerte bessa,laerte rodrigues de bessa,Flight tickets,07575651000159,1763527,774.52
296568,2010-10-12T00:00:00,lelo coimbra,welington coimbra,Flight tickets,34318100715,1944297,614.42
341245,2010-01-06T00:00:00,maia filho,jose de andrade maia filho,"Lodging, except for congressperson from Distri...",09967852007563,1775596,216.00
376127,2010-10-02T00:00:00,darcisio perondi,darcisio paulo perondi,Telecommunication,08262748034,1941983,483.59
376128,2010-10-02T00:00:00,darcisio perondi,darcisio paulo perondi,Telecommunication,08262748034,1941996,301.42


In [9]:
companies = pd.read_csv('../data/2016-09-03-companies.xz', low_memory=False)

In [10]:
companies['cnpj'].head()

0    05.939.467/0001-15
1    16.571.066/0001-71
2    07.069.926/0001-82
3    05.246.123/0001-20
4    04.606.157/0001-16
Name: cnpj, dtype: object

In [11]:
companies['cnpj'] = companies['cnpj'].str.replace(r'[\.\/\-]', '')

In [12]:
data = pd.merge(data, companies,
       how='left',
       left_on='cnpj_cpf',
       right_on='cnpj')

In [13]:
data.name = data.name.apply(normalize_string)
data.trade_name = data.trade_name.apply(normalize_string)

In [14]:
suspect_row = data.supplier.isin(deputies['civil_name']) | data.name.isin(deputies['civil_name']) | data.trade_name.isin(deputies['civil_name']) 
data[suspect_row].shape

(14, 259)

In [15]:
import re

regex = re.compile(r'(?:{})'.format('|'.join(deputies.civil_name)))
rows = data.supplier.apply(lambda name: regex.search(name) is not None)
data[rows].shape

(72, 259)

In [16]:
data.loc[rows,
         ['issue_date', 'congressperson_name', 'supplier', 'subquota_description', 'cnpj_cpf', 'document_id', 'total_net_value']]

,issue_date,congressperson_name,supplier,subquota_description,cnpj_cpf,document_id,total_net_value
73923,2009-06-06T00:00:00,afonso hamm,joao rodrigues manta hoteis de turismo ltda,"Locomotion, meal and lodging",92209790000561,1603993,207.67
73924,2009-05-26T00:00:00,afonso hamm,joao rodrigues manta hoteis de turismo ltda,"Locomotion, meal and lodging",92209790000138,1604000,101.68
73925,2009-05-26T00:00:00,afonso hamm,joao rodrigues manta hoteis de turismo ltda,"Locomotion, meal and lodging",92209790000561,1604001,129.42
74332,2009-10-03T00:00:00,afonso hamm,joao rodrigues manta joteis de turismo ltda,"Lodging, except for congressperson from Distri...",92209790000561,1689370,89.25
200406,2010-12-28T00:00:00,marcelo castro,marcelo costa e castro,Flight tickets,04114985000137,1985323,12000.00
204127,2010-08-27T00:00:00,paulo magalhaes,paulo sergio paranhos de magalhaes,"Consultancy, research and technical work",06253998000112,1919603,8900.00
215985,2010-04-23T00:00:00,pompeo de mattos,joao rodrigues manta hoteis e turismo ltda,"Lodging, except for congressperson from Distri...",92209790000561,1839872,103.70
233647,2010-07-06T00:00:00,carlos manato,carlos humberto mannato,Telecommunication,04164616000159,1893988,361.92
233652,2010-07-19T00:00:00,carlos manato,carlos humberto mannato,Automotive vehicle renting or watercraft charter,57480788720,1894078,4500.00
255746,2010-03-12T00:00:00,afonso hamm,joao rodrigues manta hoteis de turismo ltda,"Lodging, except for congressperson from Distri...",92209790000561,1803098,236.48


In [17]:
deputies[deputies.civil_name == 'joao rodrigues']

,congressperson_id,civil_name
240,160571,joao rodrigues


So far we've been searching for congresspeople putting money in others' companies. When paying for a non-relative congressperson may be legal, but could raise corruption suspicions when combined with other irregularities.

In [18]:
data = pd.merge(data, deputies, how='left')

In [19]:
d = data.head()
rows = (data['supplier'] == data['civil_name']) | (data['supplier'] == data['congressperson_name'])
data.loc[rows,
         ['issue_date', 'congressperson_name', 'supplier', 'subquota_description', 'cnpj_cpf', 'document_id', 'total_net_value']]

,issue_date,congressperson_name,supplier,subquota_description,cnpj_cpf,document_id,total_net_value
41376,2009-06-22T00:00:00,assis do couto,assis do couto,Telecommunication,76535764000143,1618678,930.52
92265,2009-08-12T00:00:00,efraim filho,efraim filho,Flight tickets,02012862000160,1674474,1049.00
92432,2009-07-25T00:00:00,elismar prado,elismar prado,Flight tickets,02428624000130,1690970,290.00
182438,2010-06-07T00:00:00,arnon bezerra,arnon bezerra,Telecommunication,01009686007904,1887507,961.66
200406,2010-12-28T00:00:00,marcelo castro,marcelo costa e castro,Flight tickets,04114985000137,1985323,12000.00
204127,2010-08-27T00:00:00,paulo magalhaes,paulo sergio paranhos de magalhaes,"Consultancy, research and technical work",06253998000112,1919603,8900.00
233647,2010-07-06T00:00:00,carlos manato,carlos humberto mannato,Telecommunication,04164616000159,1893988,361.92
233652,2010-07-19T00:00:00,carlos manato,carlos humberto mannato,Automotive vehicle renting or watercraft charter,57480788720,1894078,4500.00
295219,2010-01-25T00:00:00,laerte bessa,laerte rodrigues de bessa,Flight tickets,07575651000159,1763526,667.52
295220,2010-01-29T00:00:00,laerte bessa,laerte rodrigues de bessa,Flight tickets,07575651000159,1763527,774.52


In the previous query, many rows are false positives, given mistakes filling the form. Despite of CNPJ number is right, the supplier field was registered incorrectly, using the congressperson name.